In [3]:
import numpy as np
from qutip import *
import random
random_gen = np.random.RandomState(random.SystemRandom().randrange(2 ** 32 - 1))

def CUE(nh):
    """A Random unitary matrix distributed with Haar measure"""
    z = (random_gen.randn(nh, nh) + 1j * random_gen.randn(nh, nh)) / np.sqrt(2.0)
    q, r = np.linalg.qr(z)
    d = np.diagonal(r)
    ph = d / np.absolute(d)
    q = np.multiply(q, ph, q)
    return q

In [20]:
N=11

In [7]:
psi=np.zeros((2**N))
psi[0]=1
psi_tensor=psi.reshape(tuple([2] * N))

U=1.
for n in range(N):
    u=CUE(2)
    U=np.kron(U,u)
    psi_tensor = np.einsum(u, [N, n], psi_tensor, list(range(N)), list(range(n)) + [N] + list(range(n + 1, N)))
psi=np.einsum('ij,j->i',U,psi)
psi_tensor=psi_tensor.reshape((2**N))

print('Fidelity', np.abs(np.einsum('i,i->',psi,np.conjugate(psi_tensor)))**2)

Fidelity 1.0000000000000004


In [21]:
rho=np.zeros((2**N,2**N))
rho[0,0]=0.5
rho[1,1]=0.5
rho_tensor=rho.reshape(tuple([2] * (2*N)),order='C')


U=1.
for n in range(N):
    u=CUE(2)
    #U=np.kron(U,u)
    rho_tensor = np.einsum(u, [2*N, n], rho_tensor, list(range(2*N)), np.conjugate(u), [2*N+1,N+n], list(range(n)) + [2*N] + list(range(n + 1, N+n)) + [2*N+1]+ list(range(N+n+1,2*N)))


#rho=np.einsum('ij,jk,lk->il',U,rho,np.conjugate(U))
#rho_tensor=rho_tensor.reshape((2**N,2**N))

#print('Fidelity',fidelity(Qobj(rho),Qobj(rho_tensor)))



In [22]:
rho=np.zeros((2**N,2**N))
rho[0,0]=0.5
rho[1,1]=0.5
rho_tensor=rho.reshape(tuple([2] * (2*N)),order='C')
prob_tensor=rho.reshape(tuple([2] * (2*N)),order='C')
for n in range(N):
    u=CUE(2)
    rho_tensor = np.einsum(u, [2*N, n], rho_tensor, list(range(2*N)), np.conjugate(u), [2*N+1,N+n], list(range(n)) + [2*N] + list(range(n + 1, N+n)) + [2*N+1]+ list(range(N+n+1,2*N)))
    prob_tensor = np.einsum(u, [2*N, n], prob_tensor, list(range(N))+list(range(n+N,2*N)), np.conjugate(u), [2*N,N+n], list(range(n)) + [2*N] + list(range(n + 1, N)) + list(range(N+n+1,2*N)))

prob= prob_tensor.reshape(2**N)
rho=rho_tensor.reshape((2**N,2**N))
prob_rho = np.einsum('aa->a',rho)
print(sum(abs(prob-prob_rho)))

0.0
